  
<div class="alert alert-info">
<h1><center><font color=red> House price prediction, using XGBoost <font></center></h1>

- By Saber Saharkhiz
</div>

In [1]:
import numpy as np
import pandas as pd

import warnings   
warnings.filterwarnings('ignore')

#date of coding
import datetime
d= datetime.datetime.now()
d.strftime('%Y-%m-%d %H:%M:%S')

'2022-09-13 13:56:15'

In [2]:
# load data
data = pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# EDA:

In [3]:
print("shape of dataset: ", data.shape)

shape of dataset:  (1460, 81)


In [4]:
#Missing values?
data.isnull().sum().to_frame().T

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,0,0,259,0,0,1369,0,0,0,...,0,1453,1179,1406,0,0,0,0,0,0


In [5]:
#Check the categorical values:

def check_level(data):
    '''Checking dataset categorical features levels'''
    print("Checking the categories levels of categorical features: \n")
    for i in data.columns:
        if (data[i].dtype == object):
            print('These are the categories of \'',i,' \':\n',data[i].unique(), '\n')
    del i 
check_level(data)

Checking the categories levels of categorical features: 

These are the categories of ' MSZoning  ':
 ['RL' 'RM' 'C (all)' 'FV' 'RH'] 

These are the categories of ' Street  ':
 ['Pave' 'Grvl'] 

These are the categories of ' Alley  ':
 [nan 'Grvl' 'Pave'] 

These are the categories of ' LotShape  ':
 ['Reg' 'IR1' 'IR2' 'IR3'] 

These are the categories of ' LandContour  ':
 ['Lvl' 'Bnk' 'Low' 'HLS'] 

These are the categories of ' Utilities  ':
 ['AllPub' 'NoSeWa'] 

These are the categories of ' LotConfig  ':
 ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3'] 

These are the categories of ' LandSlope  ':
 ['Gtl' 'Mod' 'Sev'] 

These are the categories of ' Neighborhood  ':
 ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste'] 

These are the categories of ' Condition1  ':
 ['Norm' 'Feedr' 'PosN' 'Arter

In [6]:
from sklearn.impute import SimpleImputer
Imputer = SimpleImputer(missing_values=np.nan, strategy='mean')  #handling missing values

from sklearn.model_selection import train_test_split

In [7]:
data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = data.SalePrice  #Target
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

#Split data:
train_X, test_X, train_y, test_y = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.25)

NumExpr defaulting to 4 threads.


In [8]:
#using imputer to handle with missing values:
train_X = Imputer.fit_transform(train_X)
test_X = Imputer.transform(test_X)

## Modeling:

In [9]:
from xgboost import XGBRegressor  #our target is regression (SalePrice):

xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
xgb_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)